In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
d1 = pd.read_excel('/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/xgboost_v7/v7_1/V7_updated_17_aug.xlsx',sheet_name='v7_val')

In [4]:
d2 = pd.read_excel('/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/xgboost_v7/v7_1/Q1_2022.xlsx',sheet_name="Q1'22-active pool")

In [5]:
d1.head()

,OpportunityID,CreatedDate,FundedDate,OpportunityName,Priordeal_OppID,Renewal_PreviousCollectionStatus,Renewal_PreviousPace,Renewal_PreviousPaidPercentageOfRTR,Renewal_PreviousCreditScore,Renewal_CountPreviousDeals,...,WriteOff_YN,FundedWriteOffAmount_Gross,WOGross_AdvanceBalance,NetWO,Industry,Industry_Group__c,Avg_ReturnItemdays,LatestMonth_ReturnItemDays,PctChangeInReveune,PctAvgDailybalance2AvgMonthlyDeposits
0,0062M00000oPjv7QAC,2022-03-04 16:52:21,2022-03-07,Fidler Machine & Repair-Renewal Deal-03042022,0062M00000mgCZHQA2,Other,100.00,0.80,712.0,1.0,...,0.0,NaN,14900.00,NaN,Services,"75 - Automotive Repair, Services, and Parking",0.00,NaN,-0.334307,0.232447
1,0062M00000oBlSXQA0,2022-02-15 21:17:05,2022-02-17,"Law Offices of Mitchell Lee Goldfield, Esq.-Re...",0062M00000nXCpOQAW,Other,100.00,0.68,606.0,5.0,...,0.0,NaN,9247.85,NaN,Services,81 - Legal Services,0.00,NaN,-0.730462,0.971875
2,0062M00000oPQnBQAW,2022-03-01 17:42:32,2022-03-02,Discount Legal Plan-Renewal Deal-03012022,0062M00000nXGa6QAG,Default,100.00,1.09,675.0,6.0,...,0.0,NaN,19866.71,NaN,Services,81 - Legal Services,0.00,NaN,-1.462022,0.011007
3,0062M00000oPeEyQAK,2022-03-03 17:03:57,2022-03-07,Cojo Logistics-New Deal-03032022,0062M00000nXGXHQA4,Other,NaN,NaN,564.0,3.0,...,0.0,NaN,5676.40,NaN,Transportation & Public Utilities,42 - Motor Freight Transportation and Warehousing,0.00,NaN,0.425029,0.115914
4,0062M00000oBRjEQAW,2022-02-11 19:18:03,2022-02-16,Lala Restaurant-Renewal Deal-02112022,0062M00000nXI3iQAG,Default,98.82,0.99,462.0,2.0,...,0.0,NaN,115.79,NaN,Services,90 - Restaurants & Bars,1.25,2.0,-0.718831,0.015512


In [6]:
d2.head()

,OpportunityID,FundedDate,ActiveDeal_YN,WO,Funded_amount,Funded_Gross WO$,Gross_advance_balance,WO.1
0,0062M00000oAgBMQA0,2022-02-04,1,2022-09-30,11000,5377.52,10767.52,1
1,0062M00000oQI2hQAG,2022-03-16,1,NaT,7500,NaN,NaN,0
2,0062M00000oPA8rQAG,2022-02-25,1,2022-09-30,700000,449342.67,792342.67,1
3,0062M00000oQ6ltQAC,2022-03-21,1,2022-08-31,5000,828.14,3278.14,1
4,0062M00000oB6N6QAK,2022-02-24,1,NaT,15000,NaN,NaN,0


In [7]:
d1[d1['ActiveDeal_YN']==1]

,OpportunityID,CreatedDate,FundedDate,OpportunityName,Priordeal_OppID,Renewal_PreviousCollectionStatus,Renewal_PreviousPace,Renewal_PreviousPaidPercentageOfRTR,Renewal_PreviousCreditScore,Renewal_CountPreviousDeals,...,WriteOff_YN,FundedWriteOffAmount_Gross,WOGross_AdvanceBalance,NetWO,Industry,Industry_Group__c,Avg_ReturnItemdays,LatestMonth_ReturnItemDays,PctChangeInReveune,PctAvgDailybalance2AvgMonthlyDeposits
15,0062M00000oAgBMQA0,2022-02-03 23:34:12,2022-02-04,General Strategic Resources-Renewal Deal-02032022,0062M00000nYCldQAG,Default,69.32,0.69,678.0,2.0,...,0.0,NaN,10767.52,NaN,Transportation & Public Utilities,42 - Motor Freight Transportation and Warehousing,4.750000,10.0,-0.132055,0.089969
24,0062M00000oQI2hQAG,2022-03-11 17:36:04,2022-03-16,Juniors Upholstery-Renewal Deal-03112022,0062M00000oAzu1QAC,Other,100.00,0.60,529.0,1.0,...,0.0,NaN,-1134.79,NaN,Wholesale,50 - Wholesale Trade-Durable Goods,2.000000,NaN,0.044164,0.025312
32,0062M00000oPA8rQAG,2022-02-25 15:05:49,2022-02-25,Diamond Scaffold Services-Renewal Deal-02252022,0062M00000oBDNVQA4,Default,100.00,0.15,612.0,6.0,...,0.0,NaN,792342.67,NaN,Construction,17 - Construction Special Trade Contractors,0.000000,NaN,NaN,0.084400
49,0062M00000oQ6ltQAC,2022-03-09 17:30:11,2022-03-21,Southern Belle Boutique By Dona-Renewal Deal-0...,0062M00000n2i6FQAQ,Default,100.00,1.15,613.0,5.0,...,0.0,NaN,3278.14,NaN,Retail,56 - Apparel and Accessory Stores,6.333333,NaN,-2.199999,-0.043848
50,0062M00000oB6N6QAK,2022-02-08 18:04:24,2022-02-24,Krispy Krunchy Chicken-New Deal-02082022,0062M00000n2i7rQAA,Default,100.00,1.17,560.0,1.0,...,0.0,NaN,638.28,NaN,Retail,54 - Food Stores,0.000000,NaN,NaN,-0.029916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,0062M00000oBb02QAC,2022-02-14 17:15:05,2022-02-15,Northwest Structural Construction-Renewal Deal...,0062M00000oASx2QAG,Default,100.00,0.18,545.0,2.0,...,0.0,NaN,4390.10,NaN,Construction,15 - Building Construction General Contractors...,1.750000,4.0,0.097218,0.061303
481,0062M00000oPYQOQA4,2022-03-02 18:23:04,2022-03-04,Built Better Remodeling-New Deal-03022022,0062M00000lsFQ4QAM,Other,100.00,0.91,560.0,1.0,...,0.0,NaN,2465.12,NaN,Construction,15 - Building Construction General Contractors...,0.000000,NaN,-3.591591,0.085182
482,0062M00000oYJawQAG,2022-03-25 15:02:11,2022-03-29,Crand Transportation-New Deal-03252022,0062M00000lsnJBQAY,Default,100.00,1.36,572.0,3.0,...,0.0,NaN,10963.46,NaN,Transportation & Public Utilities,47 - Transportation Services,0.000000,NaN,0.539514,0.217903
485,0062M00000oBaKSQA0,2022-02-14 15:47:33,2022-02-17,LJA Metals and Products-Renewal Deal-02142022,0062M00000n16peQAA,Other,100.00,0.69,717.0,1.0,...,0.0,NaN,52055.00,NaN,Wholesale,50 - Wholesale Trade-Durable Goods,0.000000,0.0,-0.272799,0.821749


In [8]:
len(d2)

89

In [9]:
d1.columns

Index(['OpportunityID', 'CreatedDate', 'FundedDate', 'OpportunityName',
       'Priordeal_OppID', 'Renewal_PreviousCollectionStatus',
       'Renewal_PreviousPace', 'Renewal_PreviousPaidPercentageOfRTR',
       'Renewal_PreviousCreditScore', 'Renewal_CountPreviousDeals',
       'Renewal_PreviousCountBounces', 'Renewal_PreviousPosition',
       'Renewal_PreviousHBwLCF', 'Renewal_PreviousHBwoLCF',
       'Renewal_PreviousAvgMonthRevenue', 'Renewal_PreviousAvgDailyBalance',
       'Renewal_TwoWeekBounces', 'NumberOfBankStatements',
       'BankStatement_MA_OverdraftDays', 'BankStatement_MA_NumReturnItems',
       'BankStatement_Sum_NumReturnItems', 'LastBankStatement_OverdraftDays',
       'Credit_Score__c', 'Time_in_Business__c',
       'Is_this_Business_home_based__c', 'BBC_Position__c',
       'HB_with_LCF_Payment__c', 'HBwoLCF', 'Contract_HoldBack__c',
       'AvgMonthlydeposits', 'AvgDailybalance', 'FundedAmount',
       'Paybackamount', 'NewDeal_YN', 'ActiveDeal_YN', 'WriteOff_YN',


In [10]:
d2.columns

Index(['OpportunityID', 'FundedDate', 'ActiveDeal_YN', 'WO', 'Funded_amount',
       'Funded_Gross WO$', 'Gross_advance_balance', 'WO.1'],
      dtype='object')

In [11]:
c = 0
for i in d1.index:
  for j in d2.index:
    if d1['OpportunityID'][i] == d2['OpportunityID'][j]:
      d1['WriteOff_YN'][i] = d2['WO.1'][j]
      d1['FundedAmount'][i] = d2['Funded_amount'][j]
      d1['FundedWriteOffAmount_Gross'][i] = d2['Funded_Gross WO$'][j]
      d1['WOGross_AdvanceBalance'][i] = d2['Gross_advance_balance'][j]
      d1['ActiveDeal_YN'][i] = 0
      c += 1
c

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A va

89

In [12]:
d1[d1['ActiveDeal_YN']==1]

,OpportunityID,CreatedDate,FundedDate,OpportunityName,Priordeal_OppID,Renewal_PreviousCollectionStatus,Renewal_PreviousPace,Renewal_PreviousPaidPercentageOfRTR,Renewal_PreviousCreditScore,Renewal_CountPreviousDeals,...,WriteOff_YN,FundedWriteOffAmount_Gross,WOGross_AdvanceBalance,NetWO,Industry,Industry_Group__c,Avg_ReturnItemdays,LatestMonth_ReturnItemDays,PctChangeInReveune,PctAvgDailybalance2AvgMonthlyDeposits


In [13]:
d1.to_excel('/content/drive/MyDrive/LCF_Internal/Renewable_Model/EDA Approach-1/XGBoost Implementation/xgboost_v7/v7_1/Merged_validation_set.xlsx', index = False)

In [ ]:
len(d1)

488